In [1]:
from datasets import Dataset, DatasetDict
from datetime import datetime
import os

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import string
import re
from pickle import dump, load
from unicodedata import normalize

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [4]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

### Dataset

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [6]:
# def readData(lang1, lang2):
#     print("Reading lines...")

#     # Read the file and split into lines
#     lines = open('../data/wiki/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
#         read().strip().split('\n')

#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

#     # Reverse pairs, make Lang instances
#     # if reverse:
#     #     pairs = [list(reversed(p)) for p in pairs]
#     #     input_lang = Lang(lang2)
#     #     output_lang = Lang(lang1)
#     # else:
#     #     input_lang = Lang(lang1)
#     #     output_lang = Lang(lang2)

#     data = [{lang1: pairs[0], lang2: pairs[1]}for pair in pairs]

#     return data

In [7]:
# data = readData('en', 'fr')

In [8]:
from lxml import etree

def tmx_to_tabbed_txt(tmx_file, output_file):
    """Extracts English and Hawaiian translations from a TMX file and saves them as tab-separated pairs."""
    
    # Define the XML namespace for `xml:lang`
    namespaces = {'xml': 'http://www.w3.org/XML/1998/namespace'}
    
    # Parse the TMX file
    xml_tree = etree.parse(tmx_file)
    trans_units = xml_tree.findall(".//tu")

    pairs = []
    
    # Open the output file for writing
    with open(output_file, "w", encoding="utf-8") as out_file:
        # Iterate over each translation unit
        for trans_unit in trans_units:
            pair = []
            source_text = trans_unit.find(".//tuv[@xml:lang='en']/seg", namespaces)
            target_text = trans_unit.find(".//tuv[@xml:lang='fr']/seg", namespaces)

            # Write the tab-separated pair if both texts are available
            if source_text is not None and target_text is not None and source_text.text and target_text.text:
                out_file.write(f"{source_text.text}\t{target_text.text}\n")
                pair.append(source_text.text)
                pair.append(target_text.text)
                pairs.append(pair)

    return pairs

if __name__ == "__main__":
    tmx_file = "../data/wiki/en-fr.tmx"
    output_file = "../data/wiki/en-fr.txt"
    pairs = tmx_to_tabbed_txt(tmx_file, output_file)

In [9]:
pairs = [[normalizeString(sentence) for sentence in pair] for pair in pairs]

In [10]:
len(pairs)

1365840

In [34]:
cleaned_pairs = [pair for pair in pairs if (len(pair[0]) > 0 and len(pair[0]) <= 41) and (len(pair[0]) > 0 and len(pair[0]) <= 41)]
len(cleaned_pairs)

137932

In [12]:
# Select the same amount of data with tatoeba
# tatoeba_size = 135842
# random.seed(1234)
# start_index = random.randint(0, len(pairs) - tatoeba_size)
# small_pairs = pairs[start_index:start_index + tatoeba_size]

In [13]:
# len(small_pairs)

In [14]:
# small_pairs[:10]

In [35]:
data = [{'en': pair[0], 'fr': pair[1]} for pair in cleaned_pairs]
data[:10]

[{'en': 'third stream', 'fr': 'le third stream'},
 {'en': 'gil evans influence', 'fr': 'l influence de gil evans'},
 {'en': 'the horn in the spotlight', 'fr': 'le cor sous les projecteurs'},
 {'en': 'contemporary horn in jazz', 'fr': 'le cor dans le jazz contemporain'},
 {'en': 'horn in jazz', 'fr': 'utilisateur djiboun le cor dans le jazz'},
 {'en': 'murder of jewish civil rights activists',
  'fr': 'meurtre de militants juifs des droits civiques'},
 {'en': 'questioning the golden age', 'fr': 'interrogations sur l age d or'},
 {'en': 'the problem was the condescending tone',
  'fr': 'le probleme etait le ton condescendant'},
 {'en': 'press of mississippi p lindemann albert s',
  'fr': 'black jewish relations on trial leo frank and jim conley in the new south univ'},
 {'en': 'edwards brent hayes', 'fr': 'note on the text'}]

In [36]:
random.seed(seed)
random.shuffle(data)

In [37]:
# splittng data
n = len(data)
n_train = int(0.8*n)
n_val = int(0.9*n)

train_data = data[:n_train]
val_data = data[n_train:n_val]
test_data = data[n_val:]

# convert to Huggingface dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

# Create the DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [38]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['en', 'fr'],
        num_rows: 110345
    })
    validation: Dataset({
        features: ['en', 'fr'],
        num_rows: 13793
    })
    test: Dataset({
        features: ['en', 'fr'],
        num_rows: 13794
    })
})

In [39]:
train_dataset[0]

{'en': 'main personal exhibitions',
 'fr': 'principales expositions personnelles'}

In [40]:
en_nlp = spacy.load("en_core_web_sm")
fr_nlp = spacy.load("fr_core_news_sm")

In [41]:
string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

In [42]:
def tokenize_example(example, en_nlp, fr_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    fr_tokens = [token.text for token in fr_nlp.tokenizer(example["fr"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        fr_tokens = [token.lower() for token in fr_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    fr_tokens = [sos_token] + fr_tokens + [eos_token]
    return {"en_tokens": en_tokens, "fr_tokens": fr_tokens}

In [43]:
max_length = 40
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "fr_nlp": fr_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_dataset = train_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
val_dataset = val_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
test_dataset = test_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/110345 [00:00<?, ? examples/s]

Map:   0%|          | 0/13793 [00:00<?, ? examples/s]

Map:   0%|          | 0/13794 [00:00<?, ? examples/s]

In [44]:
train_dataset[0]

{'en': 'main personal exhibitions',
 'fr': 'principales expositions personnelles',
 'en_tokens': ['<sos>', 'main', 'personal', 'exhibitions', '<eos>'],
 'fr_tokens': ['<sos>', 'principales', 'expositions', 'personnelles', '<eos>']}

In [45]:
from torchtext.vocab import build_vocab_from_iterator
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = build_vocab_from_iterator(
    train_dataset["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

fr_vocab = build_vocab_from_iterator(
    train_dataset["fr_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [46]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'the', 'in', 'of', 'and', 'was', 'a']

In [47]:
fr_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'de', 'a', 'en', 'la', 'le', 'est']

In [48]:
en_vocab['the']

4

In [49]:
len(en_vocab), len(fr_vocab)

(22065, 27292)

In [50]:
assert en_vocab[unk_token] == fr_vocab[unk_token]
assert en_vocab[pad_token] == fr_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [51]:
en_vocab['The']

RuntimeError: Token The not found and default index is not set

In [52]:
en_vocab.set_default_index(unk_index)
fr_vocab.set_default_index(unk_index)

In [53]:
en_vocab['The']

0

In [54]:
en_vocab.get_itos()[0]

'<unk>'

In [55]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

[44, 418, 11111, 2487, 621]

In [56]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', 'crime', 'shows']

In [57]:
def numericalize_example(example, en_vocab, fr_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    fr_ids = fr_vocab.lookup_indices(example["fr_tokens"])
    return {"en_ids": en_ids, "fr_ids": fr_ids}

In [58]:
fn_kwargs = {"en_vocab": en_vocab, "fr_vocab": fr_vocab}

train_dataset = train_dataset.map(numericalize_example, fn_kwargs=fn_kwargs)
val_dataset = val_dataset.map(numericalize_example, fn_kwargs=fn_kwargs)
test_dataset = test_dataset.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/110345 [00:00<?, ? examples/s]

Map:   0%|          | 0/13793 [00:00<?, ? examples/s]

Map:   0%|          | 0/13794 [00:00<?, ? examples/s]

In [59]:
train_dataset[0]

{'en': 'main personal exhibitions',
 'fr': 'principales expositions personnelles',
 'en_tokens': ['<sos>', 'main', 'personal', 'exhibitions', '<eos>'],
 'fr_tokens': ['<sos>', 'principales', 'expositions', 'personnelles', '<eos>'],
 'en_ids': [2, 177, 200, 528, 3],
 'fr_ids': [2, 430, 476, 1903, 3]}

In [60]:
data_type = "torch"
format_columns = ["en_ids", "fr_ids"]

train_dataset = train_dataset.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

val_dataset = val_dataset.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_dataset = test_dataset.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [61]:
train_dataset[0]

{'en_ids': tensor([  2, 177, 200, 528,   3]),
 'fr_ids': tensor([   2,  430,  476, 1903,    3]),
 'en': 'main personal exhibitions',
 'fr': 'principales expositions personnelles',
 'en_tokens': ['<sos>', 'main', 'personal', 'exhibitions', '<eos>'],
 'fr_tokens': ['<sos>', 'principales', 'expositions', 'personnelles', '<eos>']}

### Dataloaders

In [62]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_fr_ids = [example["fr_ids"] for example in batch]
        en_lens = [example["en_ids"].shape[0] for example in batch]
        fr_lens = [example["fr_ids"].shape[0] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_fr_ids = nn.utils.rnn.pad_sequence(batch_fr_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "fr_ids": batch_fr_ids,
            "en_lens": en_lens,
            "fr_lens": fr_lens
        }
        return batch

    return collate_fn

In [63]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [64]:
batch_size = 128

train_data_loader = get_data_loader(train_dataset, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(val_dataset, batch_size, pad_index)
test_data_loader = get_data_loader(test_dataset, batch_size, pad_index)

In [65]:
len(valid_data_loader)

108

In [66]:
len(test_data_loader)

108

In [67]:
len(train_data_loader)

863

### Model

In [68]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lens):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, np.array(src_lens), enforce_sorted=False, batch_first=False)
        packed_outputs, (hidden, cell) = self.rnn(packed)
        # outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [69]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [70]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, src_lens, trg, trg_lens, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src, src_lens)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

### Training

In [71]:
input_dim = len(fr_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [72]:
input_dim, output_dim

(27292, 22065)

In [73]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(27292, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(22065, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=22065, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [74]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 31,311,153 trainable parameters


In [75]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [76]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["fr_ids"].to(device)
        src_lens = batch["fr_lens"]
        trg = batch["en_ids"].to(device)
        trg_lens = batch["en_lens"]
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, src_lens, trg, trg_lens, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        # scheduler.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [77]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["fr_ids"].to(device)
            src_lens = batch["fr_lens"]
            trg = batch["en_ids"].to(device)
            trg_lens = batch["en_lens"]
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, src_lens, trg, trg_lens, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [78]:
n_epochs = 20
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

date = datetime.today().strftime('%Y-%m-%d')
model_dir = f'../models/wiki/{date}'
# Create the directory
try:
    os.mkdir(model_dir)
    print(f"Directory '{model_dir}' created successfully.")
except FileExistsError:
    print(f"Directory '{model_dir}' already exists.")

train_lossi = []
val_lossi = []  

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )

    train_lossi.append(train_loss)
    val_lossi.append(valid_loss)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), os.path.join(model_dir, f'mt_wiki_{train_loss:7.2f}_{valid_loss:7.2f}.pt'))
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

Directory '../models/wiki/2024-12-01' already exists.


  5%|███▎                                                              | 1/20 [04:01<1:16:24, 241.30s/it]

	Train Loss:   5.830 | Train PPL: 340.323
	Valid Loss:   5.142 | Valid PPL: 170.995


 10%|██████▌                                                           | 2/20 [08:39<1:18:54, 263.04s/it]

	Train Loss:   4.958 | Train PPL: 142.254
	Valid Loss:   4.718 | Valid PPL: 111.985


...

	Train Loss:   1.897 | Train PPL:   6.664
	Valid Loss:   4.627 | Valid PPL: 102.226


 95%|█████████████████████████████████████████████████████████████▊   | 19/20 [3:11:54<12:23, 743.96s/it]

	Train Loss:   1.845 | Train PPL:   6.327
	Valid Loss:   4.665 | Valid PPL: 106.208


100%|█████████████████████████████████████████████████████████████████| 20/20 [3:24:18<00:00, 612.94s/it]

	Train Loss:   1.807 | Train PPL:   6.095
	Valid Loss:   4.690 | Valid PPL: 108.804


### Evaluating on test set

In [79]:
model.load_state_dict(torch.load(os.path.join(model_dir, f'mt_wiki.pt')))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 4.257 | Test PPL:  70.567 |


### BLEU score

In [80]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    fr_nlp,
    en_vocab,
    fr_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in fr_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = fr_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        src_len = [tensor.shape[0]]
        hidden, cell = model.encoder(tensor, src_len)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [81]:
sentence = test_data[0]["fr"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('musiques nationales', 'national songs')

In [82]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    fr_nlp,
    en_vocab,
    fr_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [83]:
translation

['<sos>', 'international', '<eos>']

In [84]:
sentence = "Bonjour."

In [85]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    fr_nlp,
    en_vocab,
    fr_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [86]:
translation

['<sos>', '<unk>', '<unk>', '<eos>']

In [87]:
translations = [
    translate_sentence(
        example["fr"],
        model,
        en_nlp,
        fr_nlp,
        en_vocab,
        fr_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_dataset)
]

100%|██████████████████████████████████████████████████████████████| 13794/13794 [04:03<00:00, 56.58it/s]


In [88]:
bleu = evaluate.load("bleu")

In [89]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["en"]] for example in test_dataset]

In [90]:
predictions[0], references[0]

('international', ['national songs'])

In [91]:
test_dataset[0]

{'en_ids': tensor([  2, 111, 593,   3]),
 'fr_ids': tensor([   2, 5012, 2367,    3]),
 'en': 'national songs',
 'fr': 'musiques nationales',
 'en_tokens': ['<sos>', 'national', 'songs', '<eos>'],
 'fr_tokens': ['<sos>', 'musiques', 'nationales', '<eos>']}

In [92]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [93]:
tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

In [94]:
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

(['international'], ['national', 'songs'])

In [95]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [96]:
results

{'bleu': 0.07782518866744352,
 'precisions': [0.2571581801388434,
  0.10498387459686492,
  0.05091039770004792,
  0.026690167443031602],
 'brevity_penalty': 1.0,
 'length_ratio': 1.186621647899026,
 'translation_length': 67126,
 'reference_length': 56569}